In [2]:
from qmratool.models import *
from django_pandas.io import read_frame

ImproperlyConfigured: Requested setting INSTALLED_APPS, but settings are not configured. You must either define the environment variable DJANGO_SETTINGS_MODULE or call settings.configure() before accessing settings.

In [3]:
read_frame(RiskAssessment.objects.all())

NameError: name 'read_frame' is not defined